In [1]:
import random
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from pathlib import Path
from keras.utils.np_utils import to_categorical
from typing import List
import cv2 as cv
# Using SMOTE for the over sampling portion.
from imblearn.over_sampling import SMOTE
import time
%matplotlib inline

In [ ]:
model_address = os.path.join(Path(os.getcwd()).parent,'Modeling\\Existing_Models\\xgboost_dino_tuned.pkl')

In [ ]:
model = pickle.load(open(model_address, 'rb'))  # horrible.
print(model.get_xgb_params())

In [2]:
# target_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\command_keys.npy')
# screenshot_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\screenshots.npy')
target_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\command_keys_retry.npy')
screenshot_address = os.path.join(Path(os.getcwd()).parent,'Window_capture\\Data\\screenshots_retry.npy')

labels = np.load(target_address)
images = np.load(screenshot_address, allow_pickle = True)
# labels=labels[1500:1700]
# images=images[1500:1700,]

print("Length Command Keys Shape: ",labels.shape)
print("Length Screenshot Shape: ",images.shape)
print("Screenshot Shape: ",images[0].shape)
print(np.unique(labels, return_counts = True))
# res_list = [i for i, value in enumerate(labels) if value == 2] # Remove Ducking
# idx = np.random.choice(res_list, 4290, replace=False) # Randomly choose X number of entries to be deleted specified as -1
# images = pd.DataFrame(images) # flatten images then converted to dataframe for easier removal of idx
# images = np.array(images.drop(images.index[idx])) # flatten images then converted to dataframe for easier removal of idx
# labels = np.delete(labels, idx)
# print(images.shape, labels.shape)
# print(np.unique(labels, return_counts = True))

Length Command Keys Shape:  (23123,)
Length Screenshot Shape:  (23123, 64800)
Screenshot Shape:  (64800,)
(array([-1, 38]), array([15758,  7365], dtype=int64))


In [ ]:
# res_list = [i for i, value in enumerate(labels) if value == -1] # Let's get rid of some -1 values.
# idx = np.random.choice(res_list, 9000, replace=False) # Randomly choose X number of entries to be deleted specified as -1
# images = pd.DataFrame(images) # flatten images then converted to dataframe for easier removal of idx
# images = np.array(images.drop(images.index[idx])) # flatten images then converted to dataframe for easier removal of idx
# labels = np.delete(labels, idx)
# print(images.shape, labels.shape)
# print(np.unique(labels, return_counts = True))

In [ ]:
# Undersample using Tomek Links
# from imblearn.under_sampling import TomekLinks
# tl = TomekLinks()
# images, labels = tl.fit_resample(images, labels)

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 101)
images, labels = smote.fit_resample(images, labels)

In [ ]:
np.unique(labels, return_counts = True)

In [3]:
# Cast -1 to 0, 38 to 1 and 40 to 2
labels[labels == -1] = 0
labels[labels == 38] = 1
labels[labels == 40] = 2

In [4]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.25)

In [5]:
np.unique(y_train, return_counts = True)

(array([0, 1]), array([11809,  5533], dtype=int64))

In [7]:
# from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [8]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest  = xgb.DMatrix(X_test, label=y_test)

In [9]:
# Convert the data to DMatrix for xgboost

# Loop through multiple thread numbers for xgboost
start_time = time.time()
# n_estimators = 100
# param = {
#           'max_depth' : 7,
#                 'eta' : 0.1,
#     'min_child_weight': 3,
#     'colsample_bytree': 0.7,
#             'subsample': 0.8,
#            'objective':'multi:softmax',
#            'num_class': 3,
#         }

n_estimators = 300
param = {
          'max_depth' : 9,
                'eta' : 0.01,
    'min_child_weight': 4,
    'colsample_bytree': 0.8,
            'subsample': 0.7,
           'objective':'binary:logistic',
#            'num_class': 2,
        }


bst = xgb.train(param,
                dtrain,
                n_estimators,
                [(dtest, 'eval'), (dtrain, 'train')],
               early_stopping_rounds = 50)
print("XGBoost (no wrapper) Time: {}s".format(time.time() - start_time))

[18:35:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	eval-logloss:0.68862	train-logloss:0.68858
[1]	eval-logloss:0.68421	train-logloss:0.68417
[2]	eval-logloss:0.67978	train-logloss:0.67975
[3]	eval-logloss:0.67552	train-logloss:0.67549
[4]	eval-logloss:0.67124	train-logloss:0.67126
[5]	eval-logloss:0.66710	train-logloss:0.66711
[6]	eval-logloss:0.66308	train-logloss:0.66309
[7]	eval-logloss:0.65914	train-logloss:0.65911
[8]	eval-logloss:0.65524	train-logloss:0.65522
[9]	eval-logloss:0.65138	train-logloss:0.65136
[10]	eval-logloss:0.64764	train-logloss:0.64758
[11]	eval-logloss:0.64394	train-logloss:0.64386
[12]	eval-logloss:0.64029	train-logloss:0.64022
[13]	eval-logloss:0.63670	train-logloss:0.63663
[14]	eval-logloss:0.63316	tr

[164]	eval-logloss:0.42712	train-logloss:0.42498
[165]	eval-logloss:0.42669	train-logloss:0.42453
[166]	eval-logloss:0.42627	train-logloss:0.42409
[167]	eval-logloss:0.42586	train-logloss:0.42365
[168]	eval-logloss:0.42546	train-logloss:0.42321
[169]	eval-logloss:0.42506	train-logloss:0.42278
[170]	eval-logloss:0.42463	train-logloss:0.42235
[171]	eval-logloss:0.42423	train-logloss:0.42193
[172]	eval-logloss:0.42384	train-logloss:0.42151
[173]	eval-logloss:0.42345	train-logloss:0.42110
[174]	eval-logloss:0.42307	train-logloss:0.42068
[175]	eval-logloss:0.42269	train-logloss:0.42029
[176]	eval-logloss:0.42231	train-logloss:0.41988
[177]	eval-logloss:0.42192	train-logloss:0.41948
[178]	eval-logloss:0.42155	train-logloss:0.41909
[179]	eval-logloss:0.42119	train-logloss:0.41870
[180]	eval-logloss:0.42084	train-logloss:0.41830
[181]	eval-logloss:0.42048	train-logloss:0.41793
[182]	eval-logloss:0.42012	train-logloss:0.41757
[183]	eval-logloss:0.41978	train-logloss:0.41720
[184]	eval-logloss:0

In [10]:
preds = np.round(bst.predict(dtest) )
acc = 1. - (np.abs(preds - y_test).sum() / y_test.shape[0])
print("Acc: {}".format(acc))
print("Prediction time --- %s seconds ---" % (time.time() - start_time))

Acc: 0.8194084068500259
Prediction time --- 1650.6954455375671 seconds ---


In [11]:
bst.predict(dtest)

array([0.29796073, 0.30754814, 0.05016711, ..., 0.33162966, 0.02563015,
       0.31021076], dtype=float32)

In [12]:
# make predictions for test data
y_hat = np.round(bst.predict(dtest))
print(f'LogReg accuracy on held-out frames = {round(accuracy_score(y_test, y_hat),4)}')

LogReg accuracy on held-out frames = 0.8194


In [13]:
# confusion_matrix(y_test, y_hat, labels=[0, 1, 2])
# target_names = ['nothing', 'up', 'down']
confusion_matrix(y_test, y_hat, labels=[0, 1])
target_names = ['nothing', 'up']
print(classification_report(y_test, y_hat, target_names=target_names))

              precision    recall  f1-score   support

     nothing       0.79      0.99      0.88      3949
          up       0.97      0.44      0.61      1832

    accuracy                           0.82      5781
   macro avg       0.88      0.72      0.75      5781
weighted avg       0.85      0.82      0.80      5781



In [14]:
pickle.dump(bst, open('Existing_Models/xgb_retry_03.pkl', 'wb'))